# 1. Normalize the source to Float32 and apply scale/offset (if present)

In [ ]:
gdal_translate -ot Float32 -a_nodata 0 -unscale ^
  "...\PFac.tif" "...\PFac_float.tif"


Done. Wrote: C:\Users\dplatero\watem-sedem-master\watem_sedem\Beamon_Creek_Boyer_River\Output\BCR.rst


# 2. Align to DTM.rst grid (nearest-neighbor, keep NoData=0)

In [ ]:
gdalwarp -r near -tap -dstnodata 0 -of GTiff ^
  -te xmin ymin xmax ymax ^
  -tr px abs(py) ^
  -ts W H ^
  -t_srs "EPSG:XXXX" ^
  "...\PFac_float.tif" "...\PFac_aligned.tif"

# 3. Convert to RST (Float32, NoData=0)

In [ ]:
gdal_translate -of RST -ot Float32 -a_nodata 0 ^
  "...\PFac_aligned.tif" "...\PFac_aligned.rst"


# 4. Clean and snap classes (Python)

## 4.1 We then cleaned non-finite/out-of-range pixels and snapped to the allowed classes {0.02, 0.2, 0.6}

In [ ]:
from osgeo import gdal
import numpy as np

in_rst  = r"...\PFac_aligned.rst"
out_rst = r"...\PFac_clean.rst"

ds = gdal.Open(in_rst, gdal.GA_ReadOnly)
arr = ds.GetRasterBand(1).ReadAsArray().astype(np.float32)

# Clean: non-finite → 0; outside [0,1] → 0 (treat as NoData)
arr[~np.isfinite(arr)] = 0
arr[(arr < 0) | (arr > 1)] = 0

# Snap to known P classes
classes = np.array([0.02, 0.2, 0.6], dtype=np.float32)
mask = arr > 0
if mask.any():
    diffs = np.abs(arr[mask, None] - classes[None, :])
    arr[mask] = classes[np.argmin(diffs, axis=1)]

drv = gdal.GetDriverByName("RST")
out = drv.Create(out_rst, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
out.SetGeoTransform(ds.GetGeoTransform())
out.SetProjection(ds.GetProjection())
b = out.GetRasterBand(1)
b.WriteArray(arr)
b.SetNoDataValue(0)
b.FlushCache()
out, ds = None, None

print("Wrote:", out_rst)


# 5. Quality checks

After each critical step:

In [ ]:
gdalinfo -stats "...\PFac_clean.rst" | ^
  Select-String -Pattern "Type","NoData","Size is","Pixel Size","Origin","Coordinate System","Minimum","Maximum","Mean","StdDev"


## 5.1 
Expect:

Type=Float32

NoData Value=0

Minimum≈0.02, Maximum≈0.6

CRS/extent/rows/cols/pixel size identical to DTM.rst

No -inf or NaN

If you ever see -9999 in stats → NoData wasn’t applied correctly.
If you see 0/1 only → you missed -unscale or forced an integer type somewhere.

# 6. Repro tip (do it in one go)

In [ ]:
REM 1) to float
gdal_translate -ot Float32 -a_nodata 0 -unscale "...\PFac.tif" "...\PFac_float.tif"

REM 2) align (nearest)
gdalwarp -r near -tap -dstnodata 0 -of GTiff ^
  -te xmin ymin xmax ymax -tr px abs(py) -ts W H -t_srs "EPSG:XXXX" ^
  "...\PFac_float.tif" "...\PFac_aligned.tif"

REM 3) to RST
gdal_translate -of RST -ot Float32 -a_nodata 0 "...\PFac_aligned.tif" "...\PFac_aligned.rst"

REM 4) clean (if needed)
gdal_calc.py -A "...\PFac_aligned.rst" --calc="where(isfinite(A)&(A>=0)&(A<=1),A,0)" ^
  --NoDataValue=0 --type=Float32 --format=RST --overwrite --outfile="...\PFac_clean.rst"
